In [8]:
import tensorflow as tf 
from tensorflow.keras import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense, LSTM, Dropout, GRU
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical
# from keras.utils.vis_utils import plot_model
import numpy as np 
import matplotlib.pyplot as plt
import pandas as pd
import random

In [9]:
train_data = pd.read_csv("./train_data.csv")
train_input = train_data['Sentence']
train_output = train_data['Transformed sentence']

print(train_input)
print(train_output)

0       udaxihhe
1       xdvxrcsn
2       bacghqta
3       rgwuwrnh
4       osizayzf
          ...   
6995    jnozsubc
6996    vpzkyffp
6997    syuftisv
6998    ynkfgztm
6999    pivsstbe
Name: Sentence, Length: 7000, dtype: object
0       fmvmfthn
1       suiaveib
2       zgvwmloh
3       lmhdulik
4       wfysmuhe
          ...   
6995    mnhmecqd
6996    qqpmoheo
6997    ibuldqib
6998    zetqlfbh
6999    plccmpbd
Name: Transformed sentence, Length: 7000, dtype: object


In [10]:
train_input=np.array([[ord(char) - ord('a')  for char in string] for string in train_input])
# print(train_input)

train_input=np.eye(26)[train_input]
# print(train_input)

temp=np.array([[ord(char) - ord('a') + 1 for char in string] for string in train_output])
insert=np.full((temp.shape[0],1),0)
train_output=np.concatenate((insert,temp),axis=1)
# print(train_output)

train_output=np.eye(27)[train_output]
# print(train_output)


In [11]:
valid_data = pd.read_csv("./eval_data.csv")
valid_input = valid_data['Sentence']
valid_output = valid_data['Transformed sentence']

print(valid_input)
print(valid_output)

0       atxwvepa
1       iaayjhyr
2       nmnqjobz
3       puembxfo
4       kpxqtnng
          ...   
1995    iwftwmhz
1996    nicffznj
1997    oekwiwhw
1998    sxfjkgtx
1999    majeiarc
Name: Sentence, Length: 2000, dtype: object
0       ldlhgjuj
1       mffolhhl
2       xjwbqnnq
3       nqxwxmtb
4       dlbdbgvx
          ...   
1995    ecxfemba
1996    pacfyaik
1997    ujzhdgqn
1998    txwhkqnu
1999    mieohaav
Name: Transformed sentence, Length: 2000, dtype: object


In [12]:
valid_input=np.array([[ord(char) - ord('a')  for char in string] for string in valid_input])
# print(valid_input)

valid_input=np.eye(26)[valid_input]
# print(valid_input)

temp=np.array([[ord(char) - ord('a') + 1  for char in string] for string in valid_output])
insert=np.full((temp.shape[0],1),0)
valid_output=np.concatenate((insert,temp),axis=1)
# print(valid_output)

valid_output=np.eye(27)[valid_output]
# print(valid_output)


In [13]:
# import numpy as np
# import tensorflow as tf
# from tensorflow.keras.layers import Input, GRU, Dense
# from tensorflow.keras.models import Model

# # Define your input and output dimensions
# n_input = 26  # Number of unique input tokens
# n_output = 27  # Number of unique output tokens

# # Define the encoder model with two layers of GRU
# encoder_inputs = Input(shape=(None, n_input))
# encoder_gru1 = GRU(256, return_sequences=True, return_state=True, dropout=0.1, recurrent_dropout=0.3)
# encoder_outputs1, encoder_state_h1 = encoder_gru1(encoder_inputs)
# encoder_gru2 = GRU(256, return_state=True, dropout=0.1, recurrent_dropout=0.3)
# encoder_outputs2, encoder_state_h2 = encoder_gru2(encoder_outputs1)
# encoder_state = [encoder_state_h1, encoder_state_h2]

# # Define the decoder model with two layers of GRU
# decoder_inputs = Input(shape=(None, n_output))
# decoder_gru1 = GRU(256, return_sequences=True, return_state=True, dropout=0.1, recurrent_dropout=0.3)
# decoder_outputs1, _ = decoder_gru1(decoder_inputs, initial_state=encoder_state_h1)
# decoder_gru2 = GRU(256, return_sequences=True, return_state=True, dropout=0.1, recurrent_dropout=0.3)
# decoder_outputs2, _ = decoder_gru2(decoder_outputs1, initial_state=encoder_state_h2)
# decoder_dense = Dense(n_output, activation='softmax')
# decoder_outputs = decoder_dense(decoder_outputs2)

# # Create the full model
# model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# # Compile the model
# model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# # Train the model (you can use your training data here)
# model.fit([train_input, train_output[:, :-1, :]], train_output[:, 1:, :], epochs=1, batch_size=64, validation_data=([valid_input, valid_output[:, :-1, :]], valid_output[:, 1:, :]))

# # Define the encoder and decoder models for inference
# encoder_model = Model(encoder_inputs, encoder_state)

# decoder_state_input_h1 = Input(shape=(256,))
# decoder_state_input_h2 = Input(shape=(256,))
# decoder_states_input = [decoder_state_input_h1, decoder_state_input_h2]
# decoder_outputs1, decoder_state_h1 = decoder_gru1(decoder_inputs, initial_state=[decoder_state_input_h1])
# decoder_outputs2, decoder_state_h2 = decoder_gru2(decoder_outputs1, initial_state=[decoder_state_input_h2])
# decoder_state = [decoder_state_h1, decoder_state_h2]
# decoder_outputs = decoder_dense(decoder_outputs2)
# decoder_model = Model([decoder_inputs] + decoder_states_input, [decoder_outputs] + decoder_state)

# # Define the Encoder and Decoder classes for prediction
# class Encoder:
#     def __init__(self, model):
#         self.model = model

#     def predict(self, input_seq):
#         input_seq = np.array([np.eye(26)[np.array([ord(c) - ord('a')  for c in input_seq])]])
#         state = self.model.predict(input_seq)
#         return state

# class Decoder:
#     def __init__(self, model):
#         self.model = model

#     def predict(self, states):
#         target_seq = np.array([np.eye(27)[[0]]])
#         output = []
#         for i in range(8):
#             output_tokens, h1, h2 = self.model.predict([target_seq] + states)
#             output_tokens = output_tokens[0, 0, :]
#             sampled_token_index = np.argmax(output_tokens)
#             output.append(chr(sampled_token_index + ord('a') - 1))
#             target_seq = np.array([np.eye(27)[[sampled_token_index]]])
#             states = [h1, h2]
#         return ''.join(output)

# # Create instances of Encoder and Decoder
# encoder = Encoder(encoder_model)
# encoder_state = encoder.predict('sqvuzung')
# decoder = Decoder(decoder_model)
# decoded_output = decoder.predict(encoder_state)
# print(decoded_output)


In [14]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, GRU, SimpleRNN, Dense
from tensorflow.keras.models import Model

# Define your input and output dimensions
n_input = 26  # Number of unique input tokens
n_output = 27  # Number of unique output tokens

# Define the encoder model with one layer as GRU and the other as SimpleRNN
encoder_inputs = Input(shape=(None, n_input))
encoder_gru = GRU(256, return_sequences=True, return_state=True, dropout=0.1, recurrent_dropout=0.3)
encoder_outputs, encoder_state_h = encoder_gru(encoder_inputs)
encoder_rnn = SimpleRNN(256, return_state=True, dropout=0.1, recurrent_dropout=0.3)
_, encoder_state_h2 = encoder_rnn(encoder_outputs)
encoder_state = [encoder_state_h, encoder_state_h2]

# Define the decoder model with one layer as GRU and the other as SimpleRNN
decoder_inputs = Input(shape=(None, n_output))
decoder_gru = GRU(256, return_sequences=True, return_state=True, dropout=0.1, recurrent_dropout=0.3)
decoder_outputs, _ = decoder_gru(decoder_inputs, initial_state=encoder_state_h)
decoder_rnn = SimpleRNN(256, return_sequences=True, return_state=True, dropout=0.1, recurrent_dropout=0.3)
decoder_outputs2, _ = decoder_rnn(decoder_outputs, initial_state=encoder_state_h2)
decoder_dense = Dense(n_output, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs2)

# Create the full model
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model (you can use your training data here)
model.fit([train_input, train_output[:, :-1, :]], train_output[:, 1:, :], epochs=100, batch_size=64, validation_data=([valid_input, valid_output[:, :-1, :]], valid_output[:, 1:, :]))



Epoch 1/100
110/110 [==============================] - 24s 102ms/step - loss: 3.3104 - accuracy: 0.0466 - val_loss: 3.2260 - val_accuracy: 0.0694
Epoch 2/100
110/110 [==============================] - 10s 88ms/step - loss: 3.2347 - accuracy: 0.0639 - val_loss: 3.1467 - val_accuracy: 0.0767
Epoch 3/100
110/110 [==============================] - 9s 86ms/step - loss: 3.1796 - accuracy: 0.0751 - val_loss: 3.0692 - val_accuracy: 0.0941
Epoch 4/100
110/110 [==============================] - 10s 87ms/step - loss: 3.1251 - accuracy: 0.0895 - val_loss: 2.9938 - val_accuracy: 0.1110
Epoch 5/100
110/110 [==============================] - 10s 87ms/step - loss: 3.0428 - accuracy: 0.1131 - val_loss: 2.8498 - val_accuracy: 0.1529
Epoch 6/100
110/110 [==============================] - 9s 86ms/step - loss: 2.9444 - accuracy: 0.1388 - val_loss: 2.7682 - val_accuracy: 0.1546
Epoch 7/100
110/110 [==============================] - 10s 87ms/step - loss: 2.9016 - accuracy: 0.1421 - val_loss: 2.7287 - val_acc

In [15]:

# Define the encoder and decoder models for inference
encoder_model = Model(encoder_inputs, encoder_state)

decoder_state_input_h1 = Input(shape=(256,))
decoder_state_input_h2 = Input(shape=(256,))
decoder_states_input = [decoder_state_input_h1, decoder_state_input_h2]
decoder_outputs1, decoder_state_h1 = decoder_gru(decoder_inputs, initial_state=[decoder_state_input_h1])
decoder_outputs2, decoder_state_h2 = decoder_rnn(decoder_outputs1, initial_state=[decoder_state_input_h2])
decoder_state = [decoder_state_h1, decoder_state_h2]
decoder_outputs = decoder_dense(decoder_outputs2)
decoder_model = Model([decoder_inputs] + decoder_states_input, [decoder_outputs] + decoder_state)

# Define the Encoder and Decoder classes for prediction
class Encoder:
    def __init__(self, model):
        self.model = model

    def predict(self, input_seq):
        input_seq = np.array([np.eye(26)[np.array([ord(c) - ord('a')  for c in input_seq])]])
        state = self.model.predict(input_seq)
        return state

class Decoder:
    def __init__(self, model):
        self.model = model

    def predict(self, states):
        target_seq = np.array([np.eye(27)[[0]]])
        output = []
        for i in range(8):
            output_tokens, h1, h2 = self.model.predict([target_seq] + states)
            output_tokens = output_tokens[0, 0, :]
            sampled_token_index = np.argmax(output_tokens)
            output.append(chr(sampled_token_index + ord('a') - 1))
            target_seq = np.array([np.eye(27)[[sampled_token_index]]])
            states = [h1, h2]
        return ''.join(output)

# Create instances of Encoder and Decoder
encoder = Encoder(encoder_model)
encoder_state = encoder.predict('sqvuzung')
decoder = Decoder(decoder_model)
decoded_output = decoder.predict(encoder_state)
print(decoded_output)

1/1 [==============================] - 0s 38ms/step
fqcdnqyx


In [17]:
import pandas as pd
import numpy as np

# Function to check how many characters match in the two strings
def check(pred: str, true: str):
    correct = 0
    for a, b in zip(pred, true):
        if a == b:
            correct += 1

    # Prediction is more than 8 letters, so penalize for every extra letter.
    correct -= max(0, len(pred) - len(true))
    correct = max(0, correct)
    return correct

# Function to score the model's performance
def evaluate(encoder, decoder):

    # # Train data
    # print("Obtaining results for training data:")
    # train_data = pd.read_csv("train_data.csv").to_numpy()
    # results = {
    #     "pred": [],
    #     "true": [],
    #     "score": [],
    # }
    # correct = [0 for _ in range(9)]
    # for x, y in train_data:
    #     pred = decoder.predict(encoder.predict(x))
    #     score = check(pred, y)
    #     results["pred"].append(pred)
    #     results["true"].append(y)
    #     results["score"].append(score)

    #     correct[score] += 1
    # print("Train dataset results:")
    # for num_chr in range(9):
    #     print(
    #         f"Number of predictions with {num_chr} correct predictions: {correct[num_chr]}"
    #     )
    # points = sum(correct[4:6]) * 0.5 + sum(correct[6:])
    # print(f"Points: {points}")
    # # Save predicitons and true sentences to inspect manually if required.
    # pd.DataFrame.from_dict(results).to_csv("results_train.csv", index=False)

    #----------------------------------------------------------------------------------

    print("Obtaining metrics for eval data:")
    eval_data = pd.read_csv("eval_data.csv").to_numpy()
    results = {
        "pred": [],
        "true": [],
        "score": [],
    }
    correct = [0 for _ in range(9)]
    for x, y in eval_data:
        pred = decoder.predict(encoder.predict(x))
        score = check(pred, y)
        results["pred"].append(pred)
        results["true"].append(y)
        results["score"].append(score)

        correct[score] += 1
    print("Eval dataset results:")
    for num_chr in range(9):
        print(
            f"Number of predictions with {num_chr} correct predictions: {correct[num_chr]}"
        )
    points = sum(correct[4:6]) * 0.5 + sum(correct[6:])
    marks = round(min(2, points / 1400 * 2) * 2) / 2  # Rounds to the nearest 0.5
    print(f"Points: {points}")
    print(f"Marks: {marks}")
    # Save predicitons and true sentences to inspect manually if required.
    pd.DataFrame.from_dict(results).to_csv("results_eval.csv", index=False)

evaluate(encoder,decoder)

Obtaining metrics for eval data:
1/1 [==============================] - 0s 27ms/step
Eval dataset results:
Number of predictions with 0 correct predictions: 6
Number of predictions with 1 correct predictions: 72
Number of predictions with 2 correct predictions: 223
Number of predictions with 3 correct predictions: 445
Number of predictions with 4 correct predictions: 521
Number of predictions with 5 correct predictions: 454
Number of predictions with 6 correct predictions: 223
Number of predictions with 7 correct predictions: 49
Number of predictions with 8 correct predictions: 7
Points: 766.5
Marks: 1.0
